In [2]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [12]:
# classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
#            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# # config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

# PREFIX = '../../input/data/'

# epoch = 4

# # dataset 바꾸기
# cfg.data.train.classes = classes
# cfg.data.train.img_prefix = PREFIX
# cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

# cfg.data.val.classes = classes
# cfg.data.val.img_prefix = PREFIX
# cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

# cfg.data.test.classes = classes
# cfg.data.test.img_prefix = PREFIX
# cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

# cfg.data.samples_per_gpu = 4

# cfg.seed=2020
# cfg.gpu_ids = [0]
# cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
# # cfg.work_dir = './work_dirs/yolov3_d53_320_273e_trash'

# cfg.model.roi_head.bbox_head.num_classes = 11
# # cfg.model.bbox_head.num_classes = 11

# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.model.train_cfg = None

# # checkpoint path
# checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [3]:
epoch = 1

classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

cfg = Config.fromfile('./configs/yolo/yolov3.py')

cfg.data.train.classes = classes  # 새롭게 추가
cfg.data.val.classes = classes
cfg.data.test.classes = classes

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/yolov3_trash'
cfg.data.samples_per_gpu = 32

cfg.model.train_cfg = None

# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [4]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-12 07:07:28,374 - mmdet - INFO - load model from: open-mmlab://darknet53


Use load_from_local loader


In [6]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 20.2 task/s, elapsed: 41s, ETA:     0s

In [7]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.34502378 324.27823 6.2349625 408.88657 92....,batch_01_vt/0021.jpg
1,0 0.39146444 233.15092 68.97652 308.695 161.02...,batch_01_vt/0028.jpg
2,0 0.37691686 250.00066 351.23566 295.51886 447...,batch_01_vt/0031.jpg
3,0 0.39208522 166.97983 24.545273 244.78177 145...,batch_01_vt/0032.jpg
4,0 0.39948517 92.30343 303.50156 130.68451 370....,batch_01_vt/0070.jpg
